In [1]:
import collections
import re
from d2l import torch as d2l

In [2]:
# d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
#                                 '090b5e7e70c295757f55df93cb0a180b9691891a')
# d2l.download('time_machine', cache_dir='../../data')

In [4]:
def read_time_machine():  #@save
    """将时间机器数据集加载到文本行的列表中"""
    with open('../../data\\timemachine.txt', 'r') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()
print(f'# 文本总行数: {len(lines)}')
print(lines[0])
print(lines[10])

# 文本总行数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


In [8]:
def tokenize(lines, token='word'):  #@save
    """将文本行拆分为单词或字符词元"""
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('错误：未知词元类型：' + token)

tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


In [9]:
class Vocab:  #@save
    """文本词表"""
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        # 按出现频率排序
        counter = count_corpus(tokens)
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1],
                                   reverse=True)
        # 未知词元的索引为0
        self.idx_to_token = ['<unk>'] + reserved_tokens
        self.token_to_idx = {token: idx
                             for idx, token in enumerate(self.idx_to_token)}
        self.idx_to_token, self.token_to_idx = [], dict()
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self):  # 未知词元的索引为0
        return 0

    @property
    def token_freqs(self):
        return self._token_freqs

def count_corpus(tokens):  #@save
    """统计词元的频率"""
    # 这里的tokens是1D列表或2D列表
    if len(tokens) == 0 or isinstance(tokens[0], list):
        # 将词元列表展平成一个列表
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

In [10]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('the', 0), ('i', 1), ('and', 2), ('of', 3), ('a', 4), ('to', 5), ('was', 6), ('in', 7), ('that', 8), ('my', 9)]


In [11]:
def load_corpus_time_machine(max_tokens=-1):  #@save
    """返回时光机器数据集的词元索引列表和词表"""
    lines = read_time_machine()
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    # 因为时光机器数据集中的每个文本行不一定是一个句子或一个段落，
    # 所以将所有文本行展平到一个列表中
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

In [12]:
corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)

(170580, 27)

In [13]:
d2l.tra

[2,
 8,
 1,
 0,
 2,
 4,
 12,
 1,
 0,
 12,
 3,
 14,
 8,
 4,
 5,
 1,
 0,
 20,
 18,
 0,
 8,
 0,
 17,
 0,
 16,
 1,
 11,
 11,
 7,
 4,
 2,
 8,
 1,
 0,
 2,
 4,
 12,
 1,
 0,
 2,
 9,
 3,
 21,
 1,
 11,
 11,
 1,
 9,
 0,
 15,
 6,
 9,
 0,
 7,
 6,
 0,
 4,
 2,
 0,
 16,
 4,
 11,
 11,
 0,
 20,
 1,
 0,
 14,
 6,
 5,
 21,
 1,
 5,
 4,
 1,
 5,
 2,
 0,
 2,
 6,
 0,
 7,
 19,
 1,
 3,
 22,
 0,
 6,
 15,
 0,
 8,
 4,
 12,
 16,
 3,
 7,
 0,
 1,
 23,
 19,
 6,
 13,
 5,
 10,
 4,
 5,
 17,
 0,
 3,
 0,
 9,
 1,
 14,
 6,
 5,
 10,
 4,
 2,
 1,
 0,
 12,
 3,
 2,
 2,
 1,
 9,
 0,
 2,
 6,
 0,
 13,
 7,
 0,
 8,
 4,
 7,
 0,
 17,
 9,
 1,
 18,
 0,
 1,
 18,
 1,
 7,
 0,
 7,
 8,
 6,
 5,
 1,
 0,
 3,
 5,
 10,
 2,
 16,
 4,
 5,
 22,
 11,
 1,
 10,
 0,
 3,
 5,
 10,
 0,
 8,
 4,
 7,
 0,
 13,
 7,
 13,
 3,
 11,
 11,
 18,
 0,
 19,
 3,
 11,
 1,
 0,
 15,
 3,
 14,
 1,
 0,
 16,
 3,
 7,
 0,
 15,
 11,
 13,
 7,
 8,
 1,
 10,
 0,
 3,
 5,
 10,
 0,
 3,
 5,
 4,
 12,
 3,
 2,
 1,
 10,
 0,
 2,
 8,
 1,
 15,
 4,
 9,
 1,
 0,
 20,
 13,
 9,
 5,
 1,
 10,
 0,
 20,
 9,
 4,